# Training and A/B test for Conversion agents - <font color='blue'> Presence</font> of sale

In [28]:
import gym, recogym
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import pandas as pd
import pickle as pkl
pd.options.mode.chained_assignment = None 
# from inspect import getsource
from recogym.evaluate_agent_sale import verify_agents_sale, display_metrics, verify_agents_sale_extended 
from tqdm import tqdm

# env_1_sale_args is a dictionary of default parameters (i.e. number of products)
from recogym import env_1_sale_args, Configuration
from recogym.agents.sale_agent import train_agents, train_timeagents
from recogym.envs.utils_sale import format_avg_result, avg_result, format_avg_result_extended, avg_result_extended


# You can overwrite environment arguments here:
env_1_sale_args['random_seed'] = 0
env_1_sale_args['num_products'] = 10
env_1_sale_args['number_of_flips'] = 10 

num_products = env_1_sale_args['num_products']
print('Number of products =',num_products)
print('Number of flips =',env_1_sale_args['number_of_flips'])
nb_flips = env_1_sale_args['number_of_flips']

# You can overwrite environment arguments here:
env_1_sale_args['random_seed'] = 42
env_1_sale_args['mu_sale'] = False 

# env_1_sale_args['kappa'] = 0.5
print('Value of kappa =',env_1_sale_args['kappa'])

# Initialize the gym 
env = gym.make('reco-gym-sale-v1')
env.init_gym(env_1_sale_args)



Number of products = 10
Number of flips = 10
Value of kappa = 0.2


In [29]:
# Repository to save pickles
data_repo = 'data_conversion/'

## Train baseline agents

#### Settings

In [30]:
## Choose number of users for training and AB test
# Number of users for the training
# env_1_sale_args['num_users'] = 6 ##tochange !!
env_1_sale_args['num_users'] = 5000
num_users = env_1_sale_args['num_users']

# Number of users for the A/B test
# env_1_sale_args['num_users_AB'] = 7 ##tochange !!
env_1_sale_args['num_users_AB'] = 5000
num_users_AB = env_1_sale_args['num_users_AB']

# Choose user features
from recogym.agents.sale_agent import CountViewsClicksFeatureProvider, CountViewsFeatureProvider, ShareViewsClicksFeatureProvider, ShareViewsFeatureProvider
vc_feature = CountViewsClicksFeatureProvider(env.config)
v_feature = CountViewsFeatureProvider(env.config)
vc_share_feature = ShareViewsClicksFeatureProvider(env.config)
v_share_feature = ShareViewsFeatureProvider(env.config)
features = {'vc':vc_feature,
           'v':v_feature,
           'vc_share':vc_share_feature,
           'v_share':v_share_feature}
feature_name = 'v_share'
feature = features[feature_name]

#### <font color='red'> Number of A/B tests</font>

In [31]:
# # Choose number of A/B tests
# num_AB_tests = 2 ##tochange !
num_AB_tests = 25 ##tochange !

#### Logs

In [32]:
agents={}
logs={}

############## Random agent
name_agent = 'rand'+str(nb_flips)
from recogym.agents import RandomAgent, random_args
random_agent = RandomAgent(Configuration(random_args))
agents[name_agent] = random_agent


try:
    logs[name_agent] = pkl.load(open(data_repo + 'data' + str(num_users) + name_agent + '.pkl','rb'))
    print('--- Logs loaded---')
except: 
    print("--- Generate logs ---")
    logs[name_agent] = deepcopy(env).generate_logs(num_users)
    print(data_repo + 'data' + str(num_users) + name_agent + '.pkl')
    pkl.dump(logs[name_agent], open(data_repo + 'data' + str(num_users) + name_agent + '.pkl','wb'))

--- Logs loaded---


#### Training for the <font color='blue'> Presence</font> of sale

##### No weights

In [33]:
name_extension = 'pres'
name_logging = name_agent
res_dict = {}
try :
    info, save_agents = pkl.load(open(data_repo+str('agents'+str(num_users)+name_logging+feature_name+name_extension+'.pkl'),'rb'))
    print('--- Trained agents loaded --- ')
except :
    print('--- Train agents ---')
    info, save_agents = train_agents(name_logging,logs,feature_name,features, num_users=num_users, 
                                     kronecker_features=True,repo = data_repo)

--- Trained agents loaded --- 


In [34]:
try:
    for i in range(num_AB_tests):
        name_extension = 'pres'+'_nb'+str(i)
        name = 'likelihood_saleclickprod_discountpres_full'
        agent_id = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension
        print(str(data_repo) +'res_'+agent_id+name+'.pkl')
        r = pkl.load(open(str(data_repo) +'res_'+agent_id+name+'.pkl',"rb"))
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r
    print('--- A/B test results loaded ---')
    
except:
    print(' --- Run A/B test --- ')
    for i in range(num_AB_tests):
        print("-------------- A/B test n°"+str(i)+'--------------')
        name_extension = 'pres'+'_nb'+str(i)
        res=verify_agents_sale_extended(
            env,
            number_of_users=num_users_AB,
            agents={
                **agents, 
                **save_agents},
            name = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension,
            seed = 0,
            same_env = False,
            repo = data_repo
        )

        r = deepcopy(res)
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r

data_conversion/res_rand106_7_v_sharepres_nb0likelihood_saleclickprod_discountpres_full.pkl
data_conversion/res_rand106_7_v_sharepres_nb1likelihood_saleclickprod_discountpres_full.pkl
--- A/B test results loaded ---


In [35]:
res_avg = avg_result_extended(res_dict)
(res_recap, res_recap_latex, 
 res_AB, res_AB_latex, 
 res_true, res_true_latex) = format_avg_result_extended(res_avg) #get dataframe & corresponding latex table

-- A/B test --


,Agent,CTR,Att Sales,Att CR,Sales,CR
0,Rand,0.790 (0.110) %,1 (0),10.06 (0.00) %,6 (1),33.97 (8.45) %
1,Click,0.680 (0.126) %,1 (0),16.03 (8.45) %,5 (0),22.01 (8.46) %
2,PVS,0.406 (0.008) %,0 (0),10.06 (0.00) %,4 (4),25.00 (12.69) %
3,PCS,0.995 (0.353) %,1 (0),22.01 (0.00) %,5 (1),27.99 (0.00) %
4,DPCS,0.839 (0.434) %,1 (0),16.03 (0.00) %,7 (0),33.97 (8.45) %


-- True metrics --


,Agent,True CTR,True PCS,True OS,True NCS,NDPC,DPCSO,DPCSN
0,Rand,0.564 (0.026) %,4.61 (0.08) %,3.58 (0.33) %,3.60 (0.30) %,0.520 (0.014) pm,0.117 (0.033) pm,0.115 (0.030) pm
1,Click,0.499 (0.023) %,6.07 (0.12) %,4.62 (0.89) %,5.28 (0.22) %,0.606 (0.040) pm,0.143 (0.071) pm,0.079 (0.006) pm
2,PVS,0.514 (0.019) %,4.11 (0.08) %,2.97 (0.11) %,2.83 (0.08) %,0.423 (0.024) pm,0.117 (0.001) pm,0.132 (0.021) pm
3,PCS,0.739 (0.011) %,5.37 (0.37) %,3.87 (1.23) %,4.49 (0.48) %,0.794 (0.042) pm,0.224 (0.130) pm,0.131 (0.019) pm
4,DPCS,0.738 (0.234) %,6.60 (0.44) %,5.27 (0.54) %,6.18 (0.76) %,0.985 (0.374) pm,0.195 (0.048) pm,0.056 (0.027) pm


In [36]:
print(res_AB_latex+'\n')
print(res_true_latex)

\begin{tabular}{llllll}
\toprule
 Agent &              CTR & Att Sales &          Att CR &  Sales &               CR \\
\midrule
  Rand &  0.790 (0.110) \% &     1 (0) &  10.06 (0.00) \% &  6 (1) &   33.97 (8.45) \% \\
 Click &  0.680 (0.126) \% &     1 (0) &  16.03 (8.45) \% &  5 (0) &   22.01 (8.46) \% \\
   PVS &  0.406 (0.008) \% &     0 (0) &  10.06 (0.00) \% &  4 (4) &  25.00 (12.69) \% \\
   PCS &  0.995 (0.353) \% &     1 (0) &  22.01 (0.00) \% &  5 (1) &   27.99 (0.00) \% \\
  DPCS &  0.839 (0.434) \% &     1 (0) &  16.03 (0.00) \% &  7 (0) &   33.97 (8.45) \% \\
\bottomrule
\end{tabular}


\begin{tabular}{llllllll}
\toprule
 Agent &         True CTR &       True PCS &        True OS &       True NCS &              NDPC &             DPCSO &             DPCSN \\
\midrule
  Rand &  0.564 (0.026) \% &  4.61 (0.08) \% &  3.58 (0.33) \% &  3.60 (0.30) \% &  0.520 (0.014) pm &  0.117 (0.033) pm &  0.115 (0.030) pm \\
 Click &  0.499 (0.023) \% &  6.07 (0.12) \% &  4.62 (0.89) \% & 

##### Weights

In [37]:
name_extension = 'presweights'
name_logging = name_agent
res_dict = {}
try :
    info, save_agents = pkl.load(open(data_repo+str('agents'+str(num_users)+name_logging+feature_name+name_extension+'.pkl'),'rb'))
    print('--- Trained agents loaded --- ')
except :
    print('--- Train agents ---')
    info, save_agents = train_agents(name_logging,logs,feature_name,features, num_users=num_users, 
                                     kronecker_features=True,weights = True, repo = data_repo)

--- Trained agents loaded --- 


In [38]:
try:
    for i in range(num_AB_tests):
        name_extension = 'presweights'+'_nb'+str(i)
        name = 'likelihood_saleclickprod_discountpresweights_full'
        agent_id = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension
        r = pkl.load(open(str(data_repo) +'res_'+agent_id+name+'.pkl',"rb"))
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r
    print('--- A/B test results loaded ---')
    
except:
    print(' --- Run A/B test --- ')
    for i in range(num_AB_tests):
        print("-------------- A/B test n°"+str(i)+'--------------')
        name_extension = 'presweights'+'_nb'+str(i)
        res=verify_agents_sale_extended(
            env,
            number_of_users=num_users_AB,
            agents={
                **agents, 
                **save_agents},
            name = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension,
            seed = 0,
            same_env = False,
            repo = data_repo
        )

        r = deepcopy(res)
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r

--- A/B test results loaded ---


In [39]:
res_avg = avg_result_extended(res_dict)
(res_recap, res_recap_latex, 
 res_AB, res_AB_latex, 
 res_true, res_true_latex) = format_avg_result_extended(res_avg) #get dataframe & corresponding latex table

-- A/B test --


,Agent,CTR,Att Sales,Att CR,Sales,CR
0,Rand,0.550 (0.257) %,1 (1),19.02 (12.68) %,6 (5),27.98 (16.91) %
1,Click,1.011 (0.072) %,1 (1),16.03 (8.45) %,10 (1),30.98 (4.23) %
2,PVS,0.300 (0.129) %,0 (0),7.10 (4.18) %,4 (5),19.02 (12.68) %
3,PCS,0.655 (0.312) %,1 (0),13.04 (4.22) %,6 (1),19.02 (4.23) %
4,DPCS,0.307 (0.190) %,1 (1),10.09 (8.40) %,10 (4),39.94 (0.00) %


-- True metrics --


,Agent,True CTR,True PCS,True OS,True NCS,NDPC,DPCSO,DPCSN
0,Rand,0.632 (0.024) %,4.26 (0.11) %,2.93 (0.65) %,3.32 (0.01) %,0.539 (0.006) pm,0.167 (0.090) pm,0.120 (0.009) pm
1,Click,0.744 (0.194) %,6.15 (0.42) %,5.19 (0.95) %,5.60 (0.53) %,0.907 (0.175) pm,0.154 (0.116) pm,0.084 (0.037) pm
2,PVS,0.568 (0.073) %,4.75 (0.76) %,3.30 (0.71) %,3.91 (1.05) %,0.546 (0.156) pm,0.165 (0.027) pm,0.093 (0.021) pm
3,PCS,0.671 (0.058) %,5.86 (0.70) %,4.03 (0.58) %,5.15 (1.22) %,0.790 (0.162) pm,0.246 (0.037) pm,0.092 (0.062) pm
4,DPCS,0.479 (0.002) %,5.94 (0.46) %,5.41 (0.43) %,5.35 (0.78) %,0.568 (0.047) pm,0.050 (0.003) pm,0.057 (0.031) pm


In [40]:
print(res_AB_latex+'\n')
print(res_true_latex)

\begin{tabular}{llllll}
\toprule
 Agent &              CTR & Att Sales &           Att CR &   Sales &               CR \\
\midrule
  Rand &  0.550 (0.257) \% &     1 (1) &  19.02 (12.68) \% &   6 (5) &  27.98 (16.91) \% \\
 Click &  1.011 (0.072) \% &     1 (1) &   16.03 (8.45) \% &  10 (1) &   30.98 (4.23) \% \\
   PVS &  0.300 (0.129) \% &     0 (0) &    7.10 (4.18) \% &   4 (5) &  19.02 (12.68) \% \\
   PCS &  0.655 (0.312) \% &     1 (0) &   13.04 (4.22) \% &   6 (1) &   19.02 (4.23) \% \\
  DPCS &  0.307 (0.190) \% &     1 (1) &   10.09 (8.40) \% &  10 (4) &   39.94 (0.00) \% \\
\bottomrule
\end{tabular}


\begin{tabular}{llllllll}
\toprule
 Agent &         True CTR &       True PCS &        True OS &       True NCS &              NDPC &             DPCSO &             DPCSN \\
\midrule
  Rand &  0.632 (0.024) \% &  4.26 (0.11) \% &  2.93 (0.65) \% &  3.32 (0.01) \% &  0.539 (0.006) pm &  0.167 (0.090) pm &  0.120 (0.009) pm \\
 Click &  0.744 (0.194) \% &  6.15 (0.42) \% &  5.19 